# Experimental Analysis

### Imports

In [70]:
from datasets import load_dataset 
import CodeSyntaxConcept.utils as utils
from CodeSyntaxConcept.tokenizer import CodeTokenizer
from CodeSyntaxConcept.parser import TreeSitterParser
import pandas as pd

### Load Samples

In [66]:
language = "python"
#number_of_samples = 2

test_set = load_dataset("code_search_net", split='test')
test_set = test_set.filter(lambda sample: True if 
                sample['language']== language 
            else False, num_proc=1)

#test_set = utils.get_random_sub_set_test_set(test_set, number_of_samples)

No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
Loading cached processed dataset at /home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27/cache-3fbe88ea46cf0b42.arrow


### Mapping Sample Concepts

In [67]:
#checkpoint loading just to use the parser in CodeTokenizer,
checkpoint = "EleutherAI/gpt-neo-125M"
parser = TreeSitterParser(CodeTokenizer.from_pretrained(checkpoint,language))

In [68]:
test_set_node_counts = [[]]*len(parser.tokenizer.node_types)
for sample in test_set:
    tree = parser.tokenizer.parser.parse(bytes(sample['whole_func_string'], "utf8"))
    for ast_element in parser.tokenizer.node_types:
        ast_element_ocurrences = []
        utils.find_nodes(tree.root_node, ast_element, ast_element_ocurrences)
        test_set_node_counts[parser.tokenizer.node_types.index(ast_element)] = test_set_node_counts[parser.tokenizer.node_types.index(ast_element)] + [len(ast_element_ocurrences)]

In [69]:
test_set_node_frequencies = [sum(ast_frequency)/len(ast_frequency) for ast_frequency in test_set_node_counts]
assert test_set_node_frequencies[180] == sum(test_set_node_counts[180])/len(test_set_node_counts[180])

In [71]:
node_frequencies_dataframe = pd.DataFrame([], columns=['ast_element', 'average'])
for node_id, node_frequency in enumerate(test_set_node_frequencies):
    node_frequencies_dataframe.loc[len(node_frequencies_dataframe.index)] = [parser.tokenizer.node_types[node_id], node_frequency]

In [74]:
node_frequencies_dataframe.to_csv('output/node_frequencies_dataframe.csv')

### Visualization

In [73]:
node_frequencies_dataframe.sort_values(by=['average'], ascending=False)

,ast_element,average
180,identifier,40.608451
69,"""",11.048250
40,.,9.292614
108,(,9.148629
183,),9.148629
...,...,...
150,expression,0.000000
152,match_statement,0.000000
163,@=,0.000000
157,case,0.000000
